In [6]:
import yaml 
import requests
import pandas as pd
import sqlite3
from tqdm import tqdm
from bs4 import BeautifulSoup
from reportlab.pdfgen.canvas import Canvas
from reportlab.lib.pagesizes import letter
conn = sqlite3.connect("data/database.db")

In [7]:
tickers = pd.read_sql_query("SELECT * FROM master_ticker", conn)["Security"].drop_duplicates().tolist()
tickers = [t.replace(" ", "_") for t in tickers]

In [8]:
def extract_text_from_wikipedia(ticker):
    URL_BASE = "https://en.wikipedia.org/wiki/"
    request = requests.get(URL_BASE + ticker)
    soup = BeautifulSoup(request.text, "html.parser")
    #Extract text from article - Cleaning task and better scraping should be performed this is just for LLM read.
    pars = []
    for p in soup.find_all('p'):
        pars.append(str(p.text))
    
    text = "".join(pars)
    canvas = Canvas("data/wikipedia/" + ticker + ".pdf")
    width, height = letter
    # Starting position on the page
    x = 50
    y = height - 50  # top of the page
    for p in pars:
        canvas.drawString(50, 50, text)
        y -= 15
        if y < 50:
            canvas.showPage()  # End current page and start a new one
            canvas.setFont("Helvetica", 12)
            y = height - 50
    canvas.save()


In [ ]:
for t in tickers:
    extract_text_from_wikipedia(t)